In [187]:
#!/usr/bin/python
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import sys
import imp
from tensorboardX import SummaryWriter
import yaml


In [201]:
var = "*_s_6_s_7*"#'*raw*'#'_L_h1'
print(var)
os.chdir('/raid/togzhan_syrymova/lump_project/scripts/spectrogram/')
config = yaml.safe_load(open("config.yaml"))
path = config['path']
print(config)

*_s_6_s_7*
{'path': '/raid/togzhan_syrymova/lump_project/data/', 'window_len': 800, 'stride_len': 80, 'num_epochs': 500, 'batch_size': 512, 'learning_rate': 0.0001, 'channels_in': 1, 'D_out': 1, 'hidden_size': 256, 'kernel_size': 15, 'stride': 3, 'dropout': 0.3, 'valid_period': 1, 'valid_size': 100, 'model_name': 'Model'}


In [203]:
os.chdir(path)
var = "*_s_6_s_7*"#'*raw*'#'_L_h1'
csv_files_train = glob.glob('df_train' + var + '.csv') # *raw*8
csv_files_test = glob.glob('test*' + var + '.csv')
csv_files_dev = glob.glob('df_dev' + var + '.csv')
print(csv_files_train)
print(csv_files_test)
print(csv_files_dev)


['df_train_s_6_s_7.csv']
(1,)
['test_pressure_data_without_s_6_s_7.csv', 'test_pressure_data_with_s_6_s_7.csv']
(2,)
['df_dev_s_6_s_7.csv']
(1,)


In [219]:
df_train_ = pd.read_csv(csv_files_train[0], sep = ',', header = None)
df_test1 = pd.read_csv(csv_files_test[0], sep = ',', header = None)
df_test2 = pd.read_csv(csv_files_test[1], sep = ',', header = None)
df_test_ = pd.concat([df_test1, df_test2], axis = 1)
df_test_ = df_test_.transpose()
df_dev_ = pd.read_csv(csv_files_dev[0], sep = ',', header = None)

if df_train_.values[1,-1].astype('float64') > 1:
    train_y = np.floor(pd.DataFrame((df_train_.values[:,-1].astype('float64')))/10)
    dev_y = np.floor(pd.DataFrame((df_dev_.values[:,-1].astype('float64')))/10)
else:
    train_y = pd.DataFrame((df_train_.values[:,-1].astype('float64')))
    dev_y = pd.DataFrame((df_dev_.values[:,-1].astype('float64')))

# Create test labels
test_y = pd.concat([ pd.DataFrame([0 for i in range(np.shape(df_test1)[1])]), pd.DataFrame([1 for i in range(np.shape(df_test2)[1])])], axis= 0)
test_y = test_y.reset_index(drop=True)
################################################################
# get data
df_train = pd.DataFrame(df_train_.values[:,:-1].astype('float64'))
df_dev = pd.DataFrame(df_dev_.values[:,:-1].astype('float64'))
df_test = df_test_.astype('float64')

print(np.shape(df_train))
print(np.shape(df_dev))
print(np.shape(df_test))

(3072, 16800)
(1536, 16800)
(1152, 16800)


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_train)

In [222]:
df_train_

,0,1,2,3,4,5,6,7,8,9,...,16791,16792,16793,16794,16795,16796,16797,16798,16799,16800
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,...,3.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,5.0
1,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11.0
2,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,14.0
3,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
4,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0
3068,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,16.0
3069,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3070,2.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,2.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,10.0


In [ ]:
3072/2